In [ ]:

!pip install numpy
!pip install matplotlib
!pip install pandas
!pip install sklearn
!pip install foolbox
!pip install tensorflow==2.14.0 tensorflow-addons==0.23.0


  Using cached sklearn-0.0.post12.tar.gz (2.6 kB)
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [ ]:
import tensorflow as tf
import tensorflow_addons as tfa

print("TensorFlow version:", tf.__version__)
print("TensorFlow Addons version:", tfa.__version__)

TensorFlow version: 2.14.0
TensorFlow Addons version: 0.23.0


/usr/local/lib/python3.11/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import numpy as np
import tensorflow as tf
import pickle

In [ ]:
# Load the data
with open('/content/drive/MyDrive/CDAC_DATASET/train.p', 'rb') as f:
    train_data = pickle.load(f)

with open('/content/drive/MyDrive/CDAC_DATASET/test.p', 'rb') as f:
    test_data = pickle.load(f)

with open('/content/drive/MyDrive/CDAC_DATASET/valid.p', 'rb') as f:
    val_data = pickle.load(f)

The signnames.csv file in the GTSRB dataset typically contains the mapping of class IDs to their corresponding sign names. This file is useful for interpreting the class labels predicted by your model (e.g., converting a numeric class ID to a human-readable traffic sign name).

To use the signnames.csv file, you need to load it and create a dictionary that maps class IDs to sign names. Here is how you can do it

In [ ]:
import pandas as pd

signnames = pd.read_csv('/content/drive/MyDrive/CDAC_DATASET/signname.csv')
class_id_to_name = dict(zip(signnames['ClassId'], signnames['SignName']))

In [ ]:
# Extract features and labels
X_train, y_train = train_data['features'], train_data['labels']
X_test, y_test = test_data['features'], test_data['labels']
X_val, y_val = val_data['features'], val_data['labels']


In [ ]:
# Normalize the images
X_train = X_train/255.0
X_test = X_test/255.0
X_val = X_val/255.0

In [ ]:
# Convert labels to categorical
y_train = tf.keras.utils.to_categorical(y_train, 43)
y_test = tf.keras.utils.to_categorical(y_test, 43)
y_val = tf.keras.utils.to_categorical(y_val, 43)

In [ ]:
# Build the ResNet model
def build_resnet_model(input_shape=(32, 32, 3), num_classes=43):
    base_model = tf.keras.applications.ResNet50(
        input_shape=input_shape,
        include_top=False,
        weights=None
    )

    x = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)
    x = tf.keras.layers.Dense(1024, activation='relu')(x)
    x = tf.keras.layers.Dense(num_classes, activation='softmax')(x)

    model = tf.keras.models.Model(inputs=base_model.input, outputs=x)

    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    return model

In [ ]:
model = build_resnet_model()
model.summary()


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 32, 32, 3)]          0         []                            
                                                                                                  
 conv1_pad (ZeroPadding2D)   (None, 38, 38, 3)            0         ['input_1[0][0]']             
                                                                                                  
 conv1_conv (Conv2D)         (None, 16, 16, 64)           9472      ['conv1_pad[0][0]']           
                                                                                                  
 conv1_bn (BatchNormalizati  (None, 16, 16, 64)           256       ['conv1_conv[0][0]']          
 on)                                                                                          

In [ ]:
# Train the model
history = model.fit(X_train, y_train,
                    epochs=10,
                    validation_data=(X_val, y_val),
                    batch_size=64)


Epoch 1/10
  1/544 [..............................] - ETA: 27:59 - loss: 0.4635 - accuracy: 0.8281

KeyboardInterrupt: 

In [ ]:
# Save the model
model.save('/path/to/save/resnet_model.h5')

In [ ]:
# Load the model
import tensorflow as tf

model = tf.keras.models.load_model('/path/to/save/resnet_model.h5')

In [ ]:
# Create adversarial examples using FGSM
def create_adversarial_pattern(model, input_image, input_label):
    with tf.GradientTape() as tape:
        tape.watch(input_image)
        prediction = model(input_image)
        loss = tf.keras.losses.categorical_crossentropy(input_label, prediction)

    gradient = tape.gradient(loss, input_image)
    signed_grad = tf.sign(gradient)
    return signed_grad

In [ ]:
def generate_adversarial_examples(model, images, labels, epsilon=0.1):
    adv_images = []
    for img, lbl in zip(images, labels):
        perturbations = create_adversarial_pattern(model, tf.convert_to_tensor([img]), tf.convert_to_tensor([lbl]))
        adv_image = img + epsilon * perturbations[0]
        adv_image = tf.clip_by_value(adv_image, 0, 1)
        adv_images.append(adv_image)
    return np.array(adv_images)

In [ ]:
X_test_adv = generate_adversarial_examples(model, X_test, y_test, epsilon=0.1)

NameError: name 'model' is not defined

In [ ]:
# Evaluate the model on adversarial examples
adv_loss, adv_acc = model.evaluate(X_test_adv, y_test)
print(f'Adversarial Test Loss: {adv_loss}, Adversarial Test Accuracy: {adv_acc}')

In [ ]:
# Adversarial training
X_train_adv = generate_adversarial_examples(model, X_train, y_train, epsilon=0.1)
X_train_combined = np.concatenate((X_train, X_train_adv))
y_train_combined = np.concatenate((y_train, y_train))

history_adv = model.fit(X_train_combined, y_train_combined,
                        epochs=10,
                        validation_data=(X_val, y_val),
                        batch_size=64)

In [ ]:
# Get the sign name from the class ID
def get_sign_name(class_id):
    return class_id_to_name.get(class_id, "Unknown")

# Example usage
predictions = model.predict(X_test)
predicted_class_ids = np.argmax(predictions, axis=1)

# Print some predictions and their corresponding sign names
for i in range(10):
    print(f"Predicted class ID: {predicted_class_ids[i]}, Sign Name: {get_sign_name(predicted_class_ids[i])}")

Explanation
1. Load the Data: Load the training, validation, and test data directly from the provided files.
2. Normalize the Images: Scale the pixel values of the images to the range [0, 1].
3. Convert Labels to Categorical: Convert the labels to one-hot encoded format for training.
4. Build the ResNet Model: Define and compile a ResNet model.
5. Train the Model: Train the model on the training data and validate on the validation data.
6. Create Adversarial Examples: Generate adversarial examples using the Fast Gradient Sign Method (FGSM).
7. Evaluate on Adversarial Examples: Evaluate the model on the adversarial examples.
8. Adversarial Training: Retrain the model using both original and adversarial examples.